In [7]:
%%sql

CREATE DATABASE IF NOT EXISTS timeseries_db;

USE timeseries_db;

CREATE TABLE IF NOT EXISTS tick (
        symbol VARCHAR(10),
        ts     DATETIME SERIES TIMESTAMP,
        open   NUMERIC(18, 2),
        high   NUMERIC(18, 2),
        low    NUMERIC(18, 2),
        price  NUMERIC(18, 2),
        volume INT,
        KEY(ts)
);

CREATE TABLE IF NOT EXISTS stock_sentiment (
        headline  VARCHAR(250),
        positive  FLOAT,
        negative  FLOAT,
        neutral   FLOAT,
        url       TEXT,
        publisher VARCHAR(30),
        ts        DATETIME,
        symbol    VARCHAR(10)
);

++
||
++
++

In [16]:
%%sql

CREATE PIPELINE tick
AS LOAD DATA KAFKA 'public-kafka.memcompute.com:9092/stockticker'
BATCH_INTERVAL 2500
INTO TABLE tick
FIELDS TERMINATED BY ','
(symbol,ts,open,high,low,price,volume);

++
||
++
++

In [23]:
%%sql
ALTER PIPELINE tick SET OFFSETS EARLIEST;

++
||
++
++

In [24]:
%%sql
TEST PIPELINE tick LIMIT 1;

symbol,ts,open,high,low,price,volume
RTN,2023-11-15 01:24:21,145.48,145.52,145.36,145.36,4111


In [28]:
%%sql
START PIPELINE tick;

++
||
++
++

In [40]:
!git clone https://github.com/VeryFatBoy/openai-whisper.git

Cloning into 'openai-whisper'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 36 (delta 11), reused 35 (delta 10), pack-reused 0
Receiving objects: 100% (36/36), 41.08 KiB | 20.54 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [43]:
!ls openai-whisper/dataset/stock_sentiment.csv

stock_sentiment.csv


In [151]:
import pandas as pd

# Load csv file as pandas dataframe
data = pd.read_csv("openai-whisper/dataset/stock_sentiment.csv", sep='\t')
data

,headline,positive,negative,neutral,url,publisher,ts,symbol
0,Agilent Technologies Announces Pricing of $5……...,0.110814,0.012814,0.876372,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A
1,Agilent (A) Gears Up for Q2 Earnings: What's i...,0.068731,0.018787,0.912482,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A
2,"Pershing Square Capital Management, L.P. Buys ...",0.093213,0.011666,0.895121,http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A
3,J.P. Morgan Asset Management Announces Liquida...,0.014791,0.848804,0.136405,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A
4,Agilent Awards Trilogy Sciences with a Golden ...,0.627051,0.011495,0.361453,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A
...,...,...,...,...,...,...,...,...
295,Tracking Dan Loeb's Third Point Portfolio - Q3...,0.028416,0.027418,0.944166,https://seekingalpha.com/article/4221202-track...,Seeking Alpha,2018-11-12 00:00:00,A
296,Illumina's Growth Shows No Signs of Slowing Down,0.924818,0.019809,0.055373,http://www.gurufocus.com/news/768939/illuminas...,GuruFocus,2018-11-12 00:00:00,A
297,Agilent Technologies (A) Earnings Expected to ...,0.220927,0.013274,0.765799,http://www.zacks.com/stock/news/337026/agilent...,Zacks,2018-11-12 00:00:00,A
298,Tracking William Von Mueffling's Cantillon Cap...,0.034504,0.018756,0.946739,https://seekingalpha.com/article/4221012-track...,Seeking Alpha,2018-11-11 00:00:00,A


In [139]:
from sqlalchemy import *
    
# create sqlalchemy engine
db_connection = create_engine(connection_url)

# Save dataframe to database, assign the tablename
data.to_sql('stock_sentiment', con = db_connection, if_exists = 'append', index=False, chunksize = 1000)

300